# packages

In [ ]:
import numpy as np

# produce matlab-style plots
import matplotlib as mpl
# increase font size on plots
mpl.rc('font',**{'size':18})
# use LaTeX to render symbols
mpl.rc('text',usetex=False)
# animation
from matplotlib import animation as ani
# Matlab-style plotting
import matplotlib.pyplot as plt

# symbolic computation, i.e. computer algebra (like Mathematica, Wolfram Alpha)
import sympy as sym

In [ ]:
# os = operating system; access OS-level commands
# e.g. create directory, delete file, execute command
# (more platform-independent than "!")
import os

# test whether this is a Colaboratory or Jupyter notebook
try:
  import google.colab
  COLAB = True
  print('Colaboratory Notebook')
except:
  COLAB = False
  print('Jupyter Notebook')

# Colab notebook
if COLAB:  
  # render SymPy equations nicely in Colaboratory Notebook
  def colab_latex_printer(exp,**options):
    from google.colab.output._publish import javascript
    url = "https://cdnjs.cloudflare.com/ajax/libs/mathjax/2.7.3/latest.js?config=default"
    javascript(url=url)
    return sym.printing.latex(exp,**options)
  
  sym.init_printing(use_latex="mathjax",latex_printer=colab_latex_printer)

# Jupyter notebook
else:
  sym.init_printing(use_latex='mathjax')

Colaboratory Notebook


# table

## variables

In [ ]:
m, I, x, y, w, dx, dy, dw, l, g, th, kappa, beta = sym.symbols(
   r'm, I, x, y, \omega, \dot{x}, \dot{y}, \dot{\omega}, \ell, g, \theta, \alpha, \beta'
    )

In [ ]:
q = sym.Matrix([[x,y,w]]).T
q


⎡  x   ⎤
⎢      ⎥
⎢  y   ⎥
⎢      ⎥
⎣\omega⎦

In [ ]:
dq = sym.Matrix([[dx,dy,dw]]).T
dq

⎡  \dot{x}   ⎤
⎢            ⎥
⎢  \dot{y}   ⎥
⎢            ⎥
⎣\dot{\omega}⎦

In [ ]:
M = sym.Matrix([[1,0,0],[0,1,0],[0,0,1]])
M

⎡1  0  0⎤
⎢       ⎥
⎢0  1  0⎥
⎢       ⎥
⎣0  0  1⎦

In [ ]:
M = sym.Matrix(sym.MatrixSymbol('M',3,3))
M

⎡M₀₀  M₀₁  M₀₂⎤
⎢             ⎥
⎢M₁₀  M₁₁  M₁₂⎥
⎢             ⎥
⎣M₂₀  M₂₁  M₂₂⎦

In [ ]:
f = sym.Matrix([[0,-g,0]]).T
f

⎡0 ⎤
⎢  ⎥
⎢-g⎥
⎢  ⎥
⎣0 ⎦

## constraints

In [ ]:
#foot locations
l1 = sym.Matrix([sym.cos(th),-sym.sin(th)])  ##right foot in body frame
l2 = sym.Matrix([-sym.cos(th), -sym.sin(th)]) ##left foot in body frame
R  = sym.Matrix([[sym.cos(w), -sym.sin(w)],[sym.sin(w), sym.cos(w)]]) ##rotation by w
p1 = R*l1 
p2 = R*l2
p1.simplify()
p2.simplify()
p1+=sym.Matrix([x,y])
p2+=sym.Matrix([x,y])

In [ ]:
# parabolic ground
a = -sym.Matrix([p1[0]**2 + p1[1], 
                 p2[0]**2 + p2[1]])
a

⎡                               2                       ⎤
⎢-y - (x + cos(\omega - \theta))  - sin(\omega - \theta)⎥
⎢                                                       ⎥
⎢                               2                       ⎥
⎣-y - (x - cos(\omega + \theta))  + sin(\omega + \theta)⎦

In [ ]:
# flat ground
#a = sym.Matrix([[(y - l*sym.sin(w+th)), (y - l*sym.sin(w-th))]]).T
#a

In [ ]:
Da = sym.Matrix.hstack(*[sym.diff(a,_) for _ in q])
Da

⎡-2⋅x - 2⋅cos(\omega - \theta)  -1  2⋅(x + cos(\omega - \theta))⋅sin(\omega - 
⎢                                                                             
⎣-2⋅x + 2⋅cos(\omega + \theta)  -1  -2⋅(x - cos(\omega + \theta))⋅sin(\omega +

\theta) - cos(\omega - \theta) ⎤
                               ⎥
 \theta) + cos(\omega + \theta)⎦

In [ ]:
Dh = sym.Matrix.hstack(Da,sym.Matrix.zeros(2,3))
Dh

⎡-2⋅x - 2⋅cos(\omega - \theta)  -1  2⋅(x + cos(\omega - \theta))⋅sin(\omega - 
⎢                                                                             
⎣-2⋅x + 2⋅cos(\omega + \theta)  -1  -2⋅(x - cos(\omega + \theta))⋅sin(\omega +

\theta) - cos(\omega - \theta)   0  0  0⎤
                                        ⎥
 \theta) + cos(\omega + \theta)  0  0  0⎦

In [ ]:
b = sym.Matrix([[x + l*sym.cos(w+th),x + l*sym.cos(w-th)]]).T
b

⎡\ell⋅cos(\omega + \theta) + x⎤
⎢                             ⎥
⎣\ell⋅cos(\omega - \theta) + x⎦

In [ ]:
Db = sym.Matrix.hstack(*[sym.diff(b,_) for _ in q])
Db

⎡1  0  -\ell⋅sin(\omega + \theta)⎤
⎢                                ⎥
⎣1  0  -\ell⋅sin(\omega - \theta)⎦

In [ ]:
Dg = sym.Matrix.hstack(Db,sym.Matrix.zeros(2,3))
Dg

⎡1  0  -\ell⋅sin(\omega + \theta)  0  0  0⎤
⎢                                         ⎥
⎣1  0  -\ell⋅sin(\omega - \theta)  0  0  0⎦

## vector fields

In [ ]:
#Mi = sym.Matrix.inv(M)
Mi = sym.Matrix.eye(3)

In [ ]:
F = {}
for b in [(-1,-1),(+1,-1),(-1,+1),(+1,+1)]:
  Mddq = f
  for j in [0,1]:
    if b[j] > 0:
      Mddq -= (kappa * a[j] + beta * (Da[j,:] * dq)[0]) * Da[j,:].T
  if b == (+1,-1):
    Mddq -= +((beta * (Da[0,:] * dq)[0])**1) * Da[0,:].T / 2 
  if b == (-1,+1):
    Mddq -= (beta * (Da[1,:] * dq)[0]) * Da[1,:].T / 2
  ddq = Mi*Mddq
  F[b] = (sym.expand(sym.Matrix.vstack(dq,ddq)))

In [ ]:
F[(-1,-1)]

⎡  \dot{x}   ⎤
⎢            ⎥
⎢  \dot{y}   ⎥
⎢            ⎥
⎢\dot{\omega}⎥
⎢            ⎥
⎢     0      ⎥
⎢            ⎥
⎢     -g     ⎥
⎢            ⎥
⎣     0      ⎦

In [ ]:
F[(+1,-1)]

⎡                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                   

In [ ]:
F[(-1,+1)]

In [ ]:
F[(+1,+1)]

## saltation matrices

In [ ]:
I6 = sym.Matrix.eye(6)

In [ ]:
M12 = ( (I6 + (F[(+1,+1)] - F[(+1,-1)])*Dh[1,:]/Dh[1,:].dot(F[(+1,-1)])) 
      * (I6 + (F[(+1,-1)] - F[(-1,-1)])*Dh[0,:]/Dh[0,:].dot(F[(-1,-1)])) )

In [ ]:
M21 = ( (I6 + (F[(+1,+1)] - F[(-1,+1)])*Dh[0,:]/Dh[0,:].dot(F[(-1,+1)])) 
      * (I6 + (F[(-1,+1)] - F[(-1,-1)])*Dh[1,:]/Dh[1,:].dot(F[(-1,-1)])) )
v=M21[-2,:].subs({x:0,w:0,dx:0,dw:0, kappa:1,beta:1, th:np.pi/4})
u=M12[-2,:].subs({x:0,w:0,dx:0,dw:0, kappa:1,beta:1, th:np.pi/4})
print(v)
print(u)


Matrix([[-1.4142135623731*(-3*\dot{y}/2 - y + 0.207106781186547)/\dot{y} + 1.4142135623731*(-\dot{y}/2 - y + 0.207106781186547)/\dot{y}, -(\dot{y}/2 + y - 0.207106781186547)/\dot{y} - (3*\dot{y}/2 + y - 0.207106781186547)/\dot{y}, -1.70710678118655*(-3*\dot{y}/2 - y + 0.207106781186547)/\dot{y} + 1.70710678118655*(-\dot{y}/2 - y + 0.207106781186547)/\dot{y}, 0, 1, 0]])
Matrix([[1.4142135623731*(-3*\dot{y}/2 - y + 0.207106781186547)/\dot{y} - 1.4142135623731*(-\dot{y}/2 - y + 0.207106781186547)/\dot{y}, -(\dot{y}/2 + y - 0.207106781186547)/\dot{y} - (3*\dot{y}/2 + y - 0.207106781186547)/\dot{y}, 1.70710678118655*(-3*\dot{y}/2 - y + 0.207106781186547)/\dot{y} - 1.70710678118655*(-\dot{y}/2 - y + 0.207106781186547)/\dot{y}, 0, 1, 0]])


In [ ]:
dM = sym.simplify((M12 - M21).subs({x:0,w:0,dx:0,dw:0}))
dM

⎡         0            0                    0                     0  0  0⎤
⎢                                                                        ⎥
⎢         0            0                    0                     0  0  0⎥
⎢                                                                        ⎥
⎢         0            0                    0                     0  0  0⎥
⎢                                                                        ⎥
⎢         0            0                    0                     0  0  0⎥
⎢                                                                        ⎥
⎢-4⋅\beta⋅cos(\theta)  0  -2⋅\beta⋅(sin(2⋅\theta) + cos(\theta))  0  0  0⎥
⎢                                                                        ⎥
⎣         0            0                    0                     0  0  0⎦

In [ ]:
sym.simplify(dM.subs({w:0}))